In [34]:
import findspark
import pyspark.sql.functions as F
from pyspark.sql.functions import round
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import mysql.connector

spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [9]:
year='2024'
month='02'
day='01'
df = spark.read.option("header", "true").csv(r"C:\Projects\Practice\Energy_Consumption_UK_split_by_post_codes.csv")

In [10]:
df2 = df.filter("LENGTH(POSTCODE) > 4")
df2 = df2.withColumn("ingestion_year", F.lit(year)).withColumn("ingestion_month", F.lit(month)).withColumn("ingestion_day", F.lit(day))
df2.show()

+--------+----------------+-----------------+----------------------+------------------------+--------------+---------------+-------------+
|POSTCODE|Number of meters|Consumption (kWh)|Mean consumption (kWh)|Median consumption (kWh)|ingestion_year|ingestion_month|ingestion_day|
+--------+----------------+-----------------+----------------------+------------------------+--------------+---------------+-------------+
|AB10 1AU|              14|          60037.7|          4288.4071429|                  3479.2|          2024|             02|           01|
|AB10 1BA|               8|          15282.9|             1910.3625|                 2051.45|          2024|             02|           01|
|AB10 1FL|              21|            46042|          2192.4761905|                  1957.7|          2024|             02|           01|
|AB10 1QH|              11|          43274.7|          3934.0636364|                  3139.7|          2024|             02|           01|
|AB10 1QS|              16|

In [12]:
df2 = df2.withColumn("ingestion_year", F.lit(year)).withColumn("ingestion_month", F.lit(month)).withColumn("ingestion_day", F.lit(day))
df2.show()

+--------+----------------+-----------------+----------------------+------------------------+--------------+---------------+-------------+
|POSTCODE|Number of meters|Consumption (kWh)|Mean consumption (kWh)|Median consumption (kWh)|ingestion_year|ingestion_month|ingestion_day|
+--------+----------------+-----------------+----------------------+------------------------+--------------+---------------+-------------+
|AB10 1AU|              14|          60037.7|          4288.4071429|                  3479.2|          2024|             02|           01|
|AB10 1BA|               8|          15282.9|             1910.3625|                 2051.45|          2024|             02|           01|
|AB10 1FL|              21|            46042|          2192.4761905|                  1957.7|          2024|             02|           01|
|AB10 1QH|              11|          43274.7|          3934.0636364|                  3139.7|          2024|             02|           01|
|AB10 1QS|              16|

In [44]:
dffinal= df2.withColumn("POSTCODE_BYAREA", F.expr("SUBSTR(POSTCODE, 0, 4)")).withColumn("Consumption_kWh", df2["Consumption (kWh)"].cast('double'))
dffinal.printSchema()
dfsum = dffinal.groupBy("POSTCODE_BYAREA").sum("Consumption_kWh")
dfsum.show(truncate = False)

root
 |-- POSTCODE: string (nullable = true)
 |-- Number of meters: string (nullable = true)
 |-- Consumption (kWh): string (nullable = true)
 |-- Mean consumption (kWh): string (nullable = true)
 |-- Median consumption (kWh): string (nullable = true)
 |-- ingestion_year: string (nullable = false)
 |-- ingestion_month: string (nullable = false)
 |-- ingestion_day: string (nullable = false)
 |-- POSTCODE_BYAREA: string (nullable = true)
 |-- Consumption_kWh: double (nullable = true)

+---------------+--------------------+
|POSTCODE_BYAREA|sum(Consumption_kWh)|
+---------------+--------------------+
|CO8            |1441945.2000000004  |
|CT17           |4257778.578065      |
|DY3            |2237945.783175      |
|IP15           |1779425.7572649997  |
|KA3            |1525747.56374       |
|DD3            |7348844.754419003   |
|DE21           |3.2220420655308988E7|
|DG16           |416330.49762100005  |
|EC1Y           |1411288.81667       |
|EH4            |1.4827090455311993E7|
|HS1 